# Loading data from a csv file, and pre-processing data for further analysis

<p style="text-align: justify">In this notebook, we will see how to perform simple and essential tasks when starting data analysis with Python. A procedural approach which is more comprehensive for beginners is used, without using the <a href="https://pandas.pydata.org/">Pandas</a> and <a href="https://www.scipy.org/">SciPy</a> packages, that are regularly used for data analysis and data scraping. This choice is done for better understanding of the different functions that are already implemented in these libraries, and to comprehend from scratch the programming behind. The different lines of code are well commented for the reader to understand. The first step in the current notebook is to import the different built-in modules in Python that are required to compute our code. </p> 

In [5]:
import csv # csv for reading the csv files
from math import sqrt # square root function from math module

### Load CSV file

<p style="text-align: justify">The data could be obtained by <a href="https://en.wikipedia.org/wiki/Data_scraping">data scrapping</a> from a website for instance, or via an <a href="https://en.wikipedia.org/wiki/Application_programming_interface">Application Programming Interface (API)</a>. </p> 

<p style="text-align: justify">Here instead, we consider loading the data from a CSV file as it is one the most common data-exchange format. A <a href="https://en.wikipedia.org/wiki/Comma-separated_values">comma separated values</a> (CSV) file contains different values separated by a delimiter, which acts as a database table or an intermediate form of a database table. In other words, a CSV file file is a set of database rows and columns stored in a text file such that the rows are separated by a new line, while the columns are separated by a semicolon or a comma. A CSV file is primarily used to transport data between two databases of different formats through a computer program.</p>

<p style="text-align: justify">The function <i>load_csv()</i> below, scrapes the data from a CSV file by reading the different rows of the file, and by storing the rows one by one in a list.</p>

In [6]:
## Load a CSV file
def load_csv(filename):
    dataset = list()  # creates a list where the row will be stored
    with open(filename, 'r') as file: # open the file in reading mode 
        csv_reader = csv.reader(file) # csv.reader built-in function
        for row in csv_reader: # loop on the rows of the file
            if not row:
                continue
            dataset.append(row) # adding row to the dataset
    return dataset

### Convert column of strings into column of floating numbers

<p style="text-align: justify">The data acquired from the function above can be of different types. The data can be made of string of characters or floating numbers for instance. When reading from a CSV file, the data is parsed as a string of character. However, in order to perform computational operations and analysis on the data, this requires that all string of numbers are transformed into floating numbers.</p>   

<p style="text-align: justify">The function <i>str_column_to_float()</i> below implements this operation for a column of the dataset. The function can then later be looped on the desired columns to turn strings into floating numbers. 
</p>

In [7]:
## Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip()) # strip() function strips all characters from the beginning and the end of the string (default whitespace characters)

### Convert column of strings into column of integers

<p style="text-align: justify">The function <i>str_column_to_float()</i> will usually be used to transform all the features of the dataset into floating numbers to perform different operations on them. However, when considering dataset for classification especially, different classes may be a string of characters as well. For instance, let's consider that in a group of person, these people are classified by age and we have 2 classes, 'Below 30' and 'Above 30'. Operations won't be performed on these classes as they will just be used for comparison with predictions, for instance in a machine learning classification problem.</p>

<p style="text-align: justify">However, transforming the data from this column into integers, can ease the classification process. For example, the string 'Below 30' is replaced by the number 0 and 'Above 30' by 1. In this way, predictions from the features are easier to compare with the classification values.</p>

<p style="text-align: justify">The function <i>str_column_to_int()</i> below allows this conversion operation for a particular column. First, a list 'class_values' containing the different rows of the column is created. From this list, 'unique' is created which is a set looking at all unique values of the list 'class_values'. A dictionary 'lookup' is then initialised to match integer values to the string of characters. The string of characters can now be referenced with their integer values, which will be usually preferred for data analysis.</p>


In [8]:
## Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset] # comprehension list with data of the considered column stored
    unique = set(class_values) # set where all unique values are stored
    lookup = dict() #
    for i, value in enumerate(unique): # enumerate function (loop + automatic counter of values)
        lookup[value] = i # value is the key of dictionary (the string of characters) and i is the number affiliated
    for row in dataset:  # modifying column in dataset
        row[column] = lookup[row[column]]
    return lookup

<p style="text-align: justify">We can now try all the different functions defined above on a particular dataset. In this notebook, we will use the <a href="https://en.wikipedia.org/wiki/Iris_flower_data_set"><i>Iris</i> flower dataset</a>, which is one of the most famous and simplest dataset available for data analysis and machine learning purposes. The dataset can be downloaded on the <a href="https://archive.ics.uci.edu/ml/">UCI Machine Learning Repository</a> website, which is the biggest open source of available data repositories for machine learning. The dataset can be downloaded at the following address <a href="https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data">https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data</a>, and needs to be saved as a CSV file.

</p>

<p style="text-align: justify">The Iris Flower Dataset involves predicting the flower species given measurements of iris flowers. It is a multiclass classification problem. The number of observations for each class is balanced. There are 150 observations with 4 input variables and 1 output variable. The variable names are as follows:</p>

1. Sepal length in cm.
2. Sepal width in cm.
3. Petal length in cm.
4. Petal width in cm.
5. Class (Iris Setosa, Iris Versicolour, Iris Virginica).

<p style="text-align: justify">The baseline performance of predicting the most prevalent class is a classification accuracy of approximately 26%. A sample of the first 5 rows is listed below.</p>


    5.1,3.5,1.4,0.2,Iris-setosa
    4.9,3.0,1.4,0.2,Iris-setosa
    4.7,3.2,1.3,0.2,Iris-setosa
    4.6,3.1,1.5,0.2,Iris-setosa
    5.0,3.6,1.4,0.2,Iris-setosa
    
<p style="text-align: justify">The piece of code below, loads the Iris dataset, and performs the different operations defined above. Some information is printed to see the modifications created on the stored dataset.</p>

In [9]:
# Load iris dataset
filename = 'iris.csv'
dataset = load_csv(filename) 

print('Loaded data file {0} with {1} rows and {2} columns'.format(filename, len(dataset), len(dataset[0])))
print('First row of the dataset: ', dataset[0]) # print first line of the dataset
print('--------------------------------------')

# convert string columns to float 
for i in range(len(dataset[0])-1): # loop on all columns except last one
    str_column_to_float(dataset, i)
# convert class column (last one) to int 
lookup = str_column_to_int(dataset, 4)

print('First row of modified dataset: ', dataset[0]) # print first line of updated dataset
print(lookup) # print lookup dictionary containing the classification of the species and their corresponding number

Loaded data file iris.csv with 150 rows and 5 columns
First row of the dataset:  ['5.1', '3.5', '1.4', '0.2', 'Iris-setosa']
--------------------------------------
First row of modified dataset:  [5.1, 3.5, 1.4, 0.2, 0]
{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}


### Normalization of set of data

<p style="text-align: justify">When it comes to analyzing data, feature scaling through normalization or standardization can be an important pre-processing step for many machine learning algorithms. While many algorithms require features to be normalized, intuitively we can think of <a href ="https://en.wikipedia.org/wiki/Principal_component_analysis">Principal Component Analysis (PCA)</a> as being a prime example of when normalization is important. PCA is a technique used for identification of a smaller number of uncorrelated variables known as principal components from a larger set of data. Indeed, when a set of data possess many features, it is important to reduce that number of features for training the models in a more efficient way. Also, data normalization makes sense when the features of a dataset are of different genders and units. 

<p style="text-align: justify">In the next part of this notebook, we consider The UCI Wine Quality Dataset, which can be directly downloaded in CSV format at <a href="https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv">https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv</a>, and it involves predicting the quality of white wines on a scale given chemical measures of each wine. It is a multiclass classification problem, but could also be framed as a regression problem. The number of observations for each class is not balanced. There are 4,898 observations with 11 input variables and 1 output variable. This dataset has continuous features that are heterogeneous in scale due to differing properties that they measure (i.e alcohol content, and malic acid). The variable names are as follows:</p>

1. Fixed acidity.
2. Volatile acidity.
3. Citric acid.
4. Residual sugar.
5. Chlorides.
6. Free sulfur dioxide.
7. Total sulfur dioxide.
8. Density.
9. pH.
10. Sulphates.
11. Alcohol.
12. Quality (score between 0 and 10).

<p style="text-align: justify">The baseline performance of predicting the mean value is a <a href="https://en.wikipedia.org/wiki/Root-mean-square_deviation">RMSE</a> of approximately 0.148 quality points. A sample of the first 5 rows is listed below.</p>

    7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8,6
    6.3,0.3,0.34,1.6,0.049,14,132,0.994,3.3,0.49,9.5,6
    8.1,0.28,0.4,6.9,0.05,30,97,0.9951,3.26,0.44,10.1,6
    7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.4,9.9,6
    7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.4,9.9,6
 
<p style="text-align: justify">As specified above, it can be interesting to normalize the features of the dataset, not only because it is required for some machine learning algorithms, but also as a way to put all data on the same scale. The first way to re-scale the data is to use the <b>Min-Max scaling</b> or more simply called "Normalization". That's usually the way we think, when talking about normalization of a dataset. It is the simplest method and consists in rescaling the range of features to scale the range in [0, 1] or [−1, 1]. Selecting the target range depends on the nature of the data. The general formula is given as</p>

$$x_{scaled} = \dfrac{x-min(x)}{max(x)-min(x)} \quad (1) \quad ,   $$ 

<p style="text-align: justify">where $x$ is the original value, $x_{scaled}$ is the normalized value.</p>

<p style="text-align: justify">From a coding point of view, it is first necessary to find the minimum and maximum of each feature (column) of the dataset, with the function <i>dataset_minmax()</i>. To do so a list variable 'minmax' is initialized, then a loop is implemented on all columns of the dataset, with the values of a column once at a time stored in the variable 'colvalues'. The minimum and maximum of 'colvalues' can then be found before adding them to the 'minmax' list and continuing the iterative loop. Finally, the 'minmax' list contains the minimum and maximum values of each column of the dataset, as this is necessary for the normalization procedure. The dataset can then be normalized by looping equation (1) over all values in the function <i>Normalize_Dataset()</i>.</p>

In [10]:
##### Normalize Data ###########

# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        colvalues = [row[i] for row in dataset]
        min_value = min(colvalues) 
        max_value = max(colvalues)
        minmax.append([min_value, max_value])
    return minmax

# Normalize the dataset except last row for classification values
def Normalize_Dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

<p style="text-align: justify">The piece of code below, loads the Wine Quality dataset, and performs the different operations defined above. The <i>load_csv()</i> function has been modified a bit to consider ';' as the delimiter and to skip the header of the dataset. Indeed, in real cases and depending on the structure of the dataset, some changes may be required to be able to import data from a CSV file. Of course, <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html">pandas.read_csv()</a> contains all these type of features, and much more, but it is interesting to see from scratch how data structure can bring some issues. Pre-processing is actually, one of if not the hardest part in machine learning. By printing the first row of the regular and normalized dataset, we can see the normalization with values between [0,1] except for the last column, which is conserved for classification purposes.</p>

In [11]:
## Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = csv.reader(file, delimiter = ";") # delimiter specified
        #Return next item in iterator to skip header
        next(csv_reader)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# load white wine quality dataset
filename = 'winequality-white.csv'
dataset = load_csv(filename) 
# convert string columns to float 
for i in range(len(dataset[0])): # loop on all columns here
    str_column_to_float(dataset, i)

print('First row of regular dataset: ', dataset[0])

# Calculate min and max for each column
minmax = dataset_minmax(dataset)

# Normalize columns
Normalize_Dataset(dataset, minmax)

print()
print('First row of normalized dataset: ', dataset[0])

First row of regular dataset:  [7.0, 0.27, 0.36, 20.7, 0.045, 45.0, 170.0, 1.001, 3.0, 0.45, 8.8, 6.0]

First row of normalized dataset:  [0.30769230769230776, 0.18627450980392157, 0.21686746987951808, 0.308282208588957, 0.10682492581602374, 0.14982578397212543, 0.37354988399071926, 0.26778484673221237, 0.25454545454545446, 0.26744186046511625, 0.12903225806451626, 6.0]


### Dataset standardization

<p style="text-align: justify">In machine learning, we can handle various types of data, e.g. audio signals and pixel values for image data, and this data can include multiple dimensions. Feature standardization makes the values of each feature in the data have zero-mean (when subtracting the mean in the numerator) and unit-variance. This method is widely used for normalization in many machine learning algorithms (e.g., <a href="https://en.wikipedia.org/wiki/Support_vector_machine">support vector machines</a>, <a href="https://en.wikipedia.org/wiki/Logistic_regression">logistic regression</a>, and <a href="https://en.wikipedia.org/wiki/Artificial_neural_network">artificial neural networks</a>). The general method of calculation is to determine the distribution <a href="https://en.wikipedia.org/wiki/Mean">mean</a> and <a href="https://en.wikipedia.org/wiki/Standard_deviation">standard deviation</a> for each feature. Next, we subtract the mean from each feature. Then we divide the values (mean is already subtracted) of each feature by its standard deviation.</p>

$$x_{std} = \dfrac{x-\mu}{\sigma} \quad (2) \quad , $$

<p style="text-align: justify">where $x_{std}$ is the standardized instance, $x$ the original instance value, $\mu$ the mean of that instance, and $\sigma$ its standard deviation.</p>

<p style="text-align: justify">In terms of programming, it requires first defining two functions <i>column_means()</i> and <i>column_stdevs()</i> that respectively calculate the means and standard deviations of the different columns of the dataset, and return the results stored in two lists 'means' and 'stdevs', respectively. The function <i>Standardize_Dataset()</i> then executes equation (2) over all instances of the dataset.</p>

In [12]:
#### Standardize Data ######

# Calculate column means
def column_means(dataset):
    means = [0 for i in range(len(dataset[0]))]
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        means[i] = sum(col_values) / float(len(dataset)) # sum of the values in column / number of elements
    return means

# Calculate column standard deviations
def column_stdevs(dataset, means):
    stdevs = [0 for i in range(len(dataset[0]))] # initialized comprehension list, number of elements = number of columns
    for i in range(len(dataset[0])):
        variance = [pow(row[i]-means[i], 2) for row in dataset] # variance with (x-mean)^2
        stdevs[i] = sum(variance)
        stdevs = [sqrt(x/(float(len(dataset)-1))) for x in stdevs] # standard deviation formula
    return stdevs

# Standardize the dataset
def Standardize_Dataset(dataset, means, stdevs):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - means[i]) / stdevs[i]

<p style="text-align: justify">The piece of code below, loads the Wine Quality dataset, and performs the different operations defined above to standardize the set of data. By printing the first row of the regular and standardized dataset, we can see the differences in the data.</p>

In [13]:
## Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = csv.reader(file, delimiter = ";") # delimiter specified
        #Return next item in iterator to skip header
        next(csv_reader)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Load white wine quality dataset
filename = 'winequality-white.csv'
dataset = load_csv(filename) 
# convert string columns to float 
for i in range(len(dataset[0])): # loop on all columns here
    str_column_to_float(dataset, i)

print('First row of regular dataset: ', dataset[0])
    
    
# Estimate mean and standard deviation
means = column_means(dataset)
stdevs = column_stdevs(dataset, means)
# standardize dataset
Standardize_Dataset(dataset, means, stdevs)

print()
print('First row of standardized dataset: ', dataset[0])

First row of regular dataset:  [7.0, 0.27, 0.36, 20.7, 0.045, 45.0, 170.0, 1.001, 3.0, 0.45, 8.8, 6.0]

First row of standardized dataset:  [708.2194955678943, -40.113082681069145, 124.81798171258234, 67353.22272363336, -3.646644578687148, 39760.87498631665, 105672.1888463459, 28.871410918992115, -403.7289363226821, -40.132468023548995, -108.13877471367468, 0.13785606532441566]
